In [1]:
import uproot
import pandas as pd

In [2]:
uproot.open("/eos/user/k/kvadla/OpenData/2lep/Data/data16.periodA-D.root")["mini"].keys()
#uproot.open("/eos/user/k/kvadla/OpenData/2lep/MC/BSM_Signal_Samples/mc15.MGPy8EG.C1N2_WZ_2L2J.root")["mini"].keys()

[b'runNumber',
 b'eventNumber',
 b'channelNumber',
 b'mcWeight',
 b'scaleFactor_PILEUP',
 b'scaleFactor_ELE',
 b'scaleFactor_MUON',
 b'scaleFactor_PHOTON',
 b'scaleFactor_TAU',
 b'scaleFactor_BTAG',
 b'scaleFactor_LepTRIGGER',
 b'scaleFactor_PhotonTRIGGER',
 b'scaleFactor_TauTRIGGER',
 b'scaleFactor_DiTauTRIGGER',
 b'trigE',
 b'trigM',
 b'trigP',
 b'trigT',
 b'trigDT',
 b'lep_n',
 b'lep_truthMatched',
 b'lep_trigMatched',
 b'lep_pt',
 b'lep_eta',
 b'lep_phi',
 b'lep_E',
 b'lep_z0',
 b'lep_charge',
 b'lep_type',
 b'lep_isTightID',
 b'lep_ptcone30',
 b'lep_etcone20',
 b'lep_trackd0pvunbiased',
 b'lep_tracksigd0pvunbiased',
 b'met_et',
 b'met_phi',
 b'jet_n',
 b'jet_pt',
 b'jet_eta',
 b'jet_phi',
 b'jet_E',
 b'jet_jvt',
 b'jet_trueflav',
 b'jet_truthMatched',
 b'jet_MV2c10',
 b'photon_n',
 b'photon_truthMatched',
 b'photon_trigMatched',
 b'photon_pt',
 b'photon_eta',
 b'photon_phi',
 b'photon_E',
 b'photon_isTightID',
 b'photon_ptcone30',
 b'photon_etcone20',
 b'photon_convType',
 b'fatje

In [3]:
l_keys = uproot.open("/eos/user/k/kvadla/OpenData/2lep/Data/data16.periodA-D.root")["mini"].keys()
#l_keys = uproot.open("/eos/user/k/kvadla/OpenData/2lep/MC/BSM_Signal_Samples/mc15.MGPy8EG.C1N2_WZ_2L2J.root")["mini"].keys()
len(l_keys)

87

In [14]:
tree = uproot.open("/eos/user/k/kvadla/OpenData/2lep/Data/data16.periodA-D.root")["mini"]
#tree = uproot.open("/eos/user/k/kvadla/OpenData/2lep/MC/BSM_Signal_Samples/mc15.MGPy8EG.C1N2_WZ_2L2J.root")["mini"]
tree

<TTree b'mini' at 0x7f1bad9a0e10>

In [15]:
df_flat = tree.pandas.df(entrystart=None, entrystop=100, flatten=None)
df_flat

,runNumber,eventNumber,channelNumber,mcWeight,scaleFactor_PILEUP,scaleFactor_ELE,scaleFactor_MUON,scaleFactor_PHOTON,scaleFactor_TAU,scaleFactor_BTAG,...,trigDT,lep_n,met_et,met_phi,jet_n,photon_n,fatjet_n,tau_n,ditau_m,met_et_syst
entry,,,,,,,,,,,,,,,,,,,,,
0,300279,109293365,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,2,11995.549805,2.722313,0,0,0,0,0.0,0.0
1,300279,109389491,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,2,20849.031250,2.458935,0,0,0,0,0.0,0.0
2,300279,109416872,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,2,5133.477539,-1.798938,0,0,0,0,0.0,0.0
3,300279,109473292,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,2,13919.626953,1.359886,0,0,0,0,0.0,0.0
4,300279,109509709,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,2,21320.576172,-1.224434,0,0,0,0,0.0,0.0
5,300279,109755668,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,2,18386.248047,-1.313833,1,0,0,0,0.0,0.0
6,300279,109715295,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,2,14726.880859,-1.786379,0,0,0,0,0.0,0.0
7,300279,109986616,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,2,6321.578125,0.198931,0,0,0,0,0.0,0.0
8,300279,111328379,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,2,17424.986328,0.995857,4,0,0,0,0.0,0.0


In [16]:
l_cuts_presel = ["trigE | trigM", "lep_n >= 2", "jet_n >= 2"]
df_flat = df_flat.query("&".join(l_cuts_presel))
df_flat

,runNumber,eventNumber,channelNumber,mcWeight,scaleFactor_PILEUP,scaleFactor_ELE,scaleFactor_MUON,scaleFactor_PHOTON,scaleFactor_TAU,scaleFactor_BTAG,...,trigDT,lep_n,met_et,met_phi,jet_n,photon_n,fatjet_n,tau_n,ditau_m,met_et_syst
entry,,,,,,,,,,,,,,,,,,,,,
0,300279,109293365,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,2,11995.549805,2.722313,0,0,0,0,0.0,0.0
1,300279,109389491,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,2,20849.031250,2.458935,0,0,0,0,0.0,0.0
2,300279,109416872,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,2,5133.477539,-1.798938,0,0,0,0,0.0,0.0
3,300279,109473292,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,2,13919.626953,1.359886,0,0,0,0,0.0,0.0
4,300279,109509709,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,2,21320.576172,-1.224434,0,0,0,0,0.0,0.0
6,300279,109715295,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,2,14726.880859,-1.786379,0,0,0,0,0.0,0.0
8,300279,111328379,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,2,17424.986328,0.995857,4,0,0,0,0.0,0.0
12,300279,111256237,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,2,59001.050781,0.952070,0,0,0,0,0.0,0.0
13,300279,110239756,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,2,13859.260742,-1.106149,0,0,0,0,0.0,0.0


In [23]:
tree.pandas.df(branches="jet_[!t]*", entrystart=None, entrystop=100, flatten=True)

jet_n         jet_pt   jet_eta   jet_phi          jet_E  \
entry subentry                                                            
5     0             1   22169.650391 -0.847633  2.599247   30840.964844   
8     0             4   36898.394531  0.550705  0.304045   42991.796875   
      1             4   29747.564453  2.376008  0.971062  161518.421875   
      2             4   25440.619141 -1.646304  2.523455   68536.265625   
      3             4   22494.291016  0.692986 -1.017262   28528.664062   
10    0             1   27408.160156 -1.926784  0.861534   96166.234375   
17    0             2   40487.335938  2.200428 -0.017894  185076.312500   
      1             2   33965.273438  0.968904 -2.452864   51396.187500   
20    0             3   31287.910156 -1.214480 -0.145510   57715.082031   
      1             3   27851.033203 -1.461987  1.483080   63315.273438   
      2             3   23786.033203  0.580231  0.203896   28221.726562   
23    0             3   31111.964844 -1.349445 -1.565981   64135.796875   
      1             3   27463.341797 -0.746224  3.086593   35612.277344   
      2             3   23984.333984 -0.857041 -2.339175   33593.308594   
24    0             1   25601.757812  1.294761  2.921680   50517.992188   
26    0             2   95023.507812 -0.350044  2.223678  101834.773438   
      1             2   71066.960938 -1.916904 -0.670391  246947.171875   
27    0             1   26445.099609  1.663570  2.008030   72429.507812   
28    0             1   23489.951172 -1.181976  2.946266   42086.941406   
31    0             1   23687.062500 -1.173834  0.847239   42182.851562   
33    0             3   55880.742188 -2.088929 -1.788439  229206.109375   
      1             3   43618.980469 -0.667880 -1.358864   54187.625000   
      2             3   28744.433594 -0.956910 -1.800434   43297.648438   
36    0             1  100302.960938  1.204788 -3.046590  183193.406250   
38    0             1   23119.761719  1.905366 -2.053745   79504.140625   
40    0             1   22097.322266  2.326669 -1.903002  114280.843750   
41    0             3   78914.656250 -2.450616 -1.704743  460964.000000   
      1             3   41385.359375  1.199430  0.252038   75402.101562   
      2             3   31313.925781 -1.668477 -3.005869   86136.804688   
43    0             6  173087.406250  0.681849 -2.619515  216361.531250   
...               ...            ...       ...       ...            ...   
      2             6   31843.884766  2.332208 -1.673628  165684.984375   
      3             6   28130.654297 -1.076319 -1.328768   46444.265625   
      4             6   25893.626953 -1.652021  1.319842   70300.242188   
      5             6   23357.904297  1.610578 -1.785915   60889.699219   
49    0             2   28517.939453  1.893127  1.469768   96881.687500   
      1             2   22407.783203  1.701767 -1.344949   63582.304688   
63    0             3  135375.750000  0.026064 -0.372457  136077.281250   
      1             3   33534.386719 -0.321212  1.453234   35426.750000   
      2             3   20556.154297  0.467680  1.439619   23432.037109   
64    0             2   22361.070312  1.360551  0.129800   46828.437500   
      1             2   20966.240234 -0.831222  2.865985   29060.105469   
67    0             2   45874.601562 -1.041750  0.187601   73313.328125   
      1             2   21863.039062  1.318588  2.954294   44060.953125   
70    0             2   28037.906250  0.703536  1.316194   35558.304688   
      1             2   23806.464844 -0.178018 -0.511163   24616.562500   
72    0             1   43361.703125 -2.485660 -1.186491  262241.437500   
77    0             1   84163.781250 -2.198688 -2.737468  384450.406250   
79    0             1   28130.521484  2.276839  0.588568  138578.296875   
81    0             1  115614.976562 -0.298780  0.097640  121497.367188   
82    0             1   24870.957031 -0.279457  0.781284   25944.287109   
83    0             1   57617.574219  2.1

In [52]:
df_jet = tree.pandas.df(branches="/jet_(?!tru).*/i", entrystart=None, entrystop=100, flatten=True)
#df_jet = tree.pandas.df(branches="jet*", entrystart=None, entrystop=100, flatten=True)
df_jet = df_jet.loc[:,"jet_pt":]
#df_jet.unstack()
df_jet

jet_pt   jet_eta   jet_phi          jet_E   jet_jvt  \
entry subentry                                                               
5     0          22169.650391 -0.847633  2.599247   30840.964844  0.721254   
8     0          36898.394531  0.550705  0.304045   42991.796875  0.970670   
      1          29747.564453  2.376008  0.971062  161518.421875  0.986996   
      2          25440.619141 -1.646304  2.523455   68536.265625  0.994803   
      3          22494.291016  0.692986 -1.017262   28528.664062  0.952519   
10    0          27408.160156 -1.926784  0.861534   96166.234375  0.735187   
17    0          40487.335938  2.200428 -0.017894  185076.312500  0.994680   
      1          33965.273438  0.968904 -2.452864   51396.187500  0.999721   
20    0          31287.910156 -1.214480 -0.145510   57715.082031  1.000000   
      1          27851.033203 -1.461987  1.483080   63315.273438  1.000000   
      2          23786.033203  0.580231  0.203896   28221.726562  0.950372   
23    0          31111.964844 -1.349445 -1.565981   64135.796875  0.971739   
      1          27463.341797 -0.746224  3.086593   35612.277344  0.997288   
      2          23984.333984 -0.857041 -2.339175   33593.308594  0.952598   
24    0          25601.757812  1.294761  2.921680   50517.992188  0.929822   
26    0          95023.507812 -0.350044  2.223678  101834.773438  0.000000   
      1          71066.960938 -1.916904 -0.670391  246947.171875  0.000000   
27    0          26445.099609  1.663570  2.008030   72429.507812  0.987950   
28    0          23489.951172 -1.181976  2.946266   42086.941406  0.991418   
31    0          23687.062500 -1.173834  0.847239   42182.851562  0.992751   
33    0          55880.742188 -2.088929 -1.788439  229206.109375  0.991612   
      1          43618.980469 -0.667880 -1.358864   54187.625000  0.999791   
      2          28744.433594 -0.956910 -1.800434   43297.648438  0.962291   
36    0         100302.960938  1.204788 -3.046590  183193.406250  0.994884   
38    0          23119.761719  1.905366 -2.053745   79504.140625  0.998021   
40    0          22097.322266  2.326669 -1.903002  114280.843750  1.000000   
41    0          78914.656250 -2.450616 -1.704743  460964.000000  1.000000   
      1          41385.359375  1.199430  0.252038   75402.101562  0.966173   
      2          31313.925781 -1.668477 -3.005869   86136.804688  1.000000   
43    0         173087.406250  0.681849 -2.619515  216361.531250  0.992151   
...                       ...       ...       ...            ...       ...   
      2          31843.884766  2.332208 -1.673628  165684.984375  0.997673   
      3          28130.654297 -1.076319 -1.328768   46444.265625  0.999093   
      4          25893.626953 -1.652021  1.319842   70300.242188  0.888347   
      5          23357.904297  1.610578 -1.785915   60889.699219  0.856652   
49    0          28517.939453  1.893127  1.469768   96881.687500  0.939988   
      1          22407.783203  1.701767 -1.344949   63582.304688  0.999847   
63    0         135375.750000  0.026064 -0.372457  136077.281250  0.994360   
      1          33534.386719 -0.321212  1.453234   35426.750000  0.843760   
      2          20556.154297  0.467680  1.439619   23432.037109  0.998234   
64    0          22361.070312  1.360551  0.129800   46828.437500  0.976144   
      1          20966.240234 -0.831222  2.865985   29060.105469  0.902361   
67    0          45874.601562 -1.041750  0.187601   73313.328125  0.999632   
      1          21863.039062  1.318588  2.954294   44060.953125  0.966715   
70    0          28037.906250  0.703536  1.316194   35558.304688  0.981539   
      1          23806.464844 -0.178018 -0.511163   24616.562500  0.946258   
72    0          43361.703125 -2.485660 -1.186491  262241.437500  0.851009   
77    0          84163.781250 -2.198688 -2.737468  384450.406250  0.990018   
79    0          28130.521484  2.276839  0.588568  138578.296875  0.884460   
81    0         115614.976562 -0.298780  0.09

In [25]:
l_cuts_jet = ["jet_pt > 30e3", "abs(jet_eta) < 2.8"] 
l_cuts_bjet = ["jet_MV2c10 < 0.6459"] # 77% WP: 0.6459; 85% WP: 0.1758
df_jet = df_jet.astype({"jet_eta":float})
df_jet = df_jet.query("&".join(l_cuts_jet + l_cuts_bjet))
df_jet

jet_pt   jet_eta   jet_phi          jet_E   jet_jvt  \
entry subentry                                                               
8     0          36898.394531  0.550705  0.304045   42991.796875  0.970670   
17    0          40487.335938  2.200428 -0.017894  185076.312500  0.994680   
      1          33965.273438  0.968904 -2.452864   51396.187500  0.999721   
23    0          31111.964844 -1.349445 -1.565981   64135.796875  0.971739   
26    0          95023.507812 -0.350044  2.223678  101834.773438  0.000000   
      1          71066.960938 -1.916904 -0.670391  246947.171875  0.000000   
33    0          55880.742188 -2.088929 -1.788439  229206.109375  0.991612   
36    0         100302.960938  1.204788 -3.046590  183193.406250  0.994884   
41    0          78914.656250 -2.450616 -1.704743  460964.000000  1.000000   
      1          41385.359375  1.199430  0.252038   75402.101562  0.966173   
      2          31313.925781 -1.668477 -3.005869   86136.804688  1.000000   
43    1          45823.367188  0.184055 -2.385459   47038.800781  0.979666   
      2          31843.884766  2.332208 -1.673628  165684.984375  0.997673   
63    0         135375.750000  0.026064 -0.372457  136077.281250  0.994360   
      1          33534.386719 -0.321212  1.453234   35426.750000  0.843760   
67    0          45874.601562 -1.041750  0.187601   73313.328125  0.999632   
72    0          43361.703125 -2.485660 -1.186491  262241.437500  0.851009   
77    0          84163.781250 -2.198688 -2.737468  384450.406250  0.990018   
81    0         115614.976562 -0.298780  0.097640  121497.367188  0.994769   
83    0          57617.574219  2.146448  2.642318  249961.656250  0.990000   
86    0          59527.644531 -0.033204 -3.062900   60078.023438  1.000000   
93    0          89549.429688  1.644423  1.331168  240621.234375  0.994335   
94    0          64267.941406 -0.071947  2.100348   65219.921875  0.955104   
96    0          49015.898438 -0.536002 -0.573045   56361.523438  0.989421   

                jet_MV2c10  jet_pt_syst  
entry subentry                           
8     0          -0.948231          0.0  
17    0          -0.798204          0.0  
      1          -0.660267          0.0  
23    0          -0.867009          0.0  
26    0          -0.702755          0.0  
      1          -0.840732          0.0  
33    0          -0.594900          0.0  
36    0          -0.322784          0.0  
41    0          -0.231897          0.0  
      1           0.485904          0.0  
      2          -0.867099          0.0  
43    1          -0.942829          0.0  
      2          -0.847150          0.0  
63    0          -0.880990          0.0  
      1          -0.926673          0.0  
67    0          -0.688472          0.0  
72    0          -0.830822          0.0  
77    0          -0.416603          0.0  
81    0          -0.647953          0.0  
83    0          -0.294525          0.0  
86    0          -0.907920          0.0  
93    0          -0.897334          0.0  
94    0          -0.092026          0.0  
96    0          -0.695699          0.0

In [26]:
n_jet_sig = df_jet.groupby(level=0).size().values
df_jet = df_jet.unstack()
df_jet = df_jet[n_jet_sig == 2]
df_jet

jet_pt                               jet_eta            \
subentry              0             1             2         0         1   
entry                                                                     
17         40487.335938  33965.273438           NaN  2.200428  0.968904   
26         95023.507812  71066.960938           NaN -0.350044 -1.916904   
43                  NaN  45823.367188  31843.884766       NaN  0.184055   
63        135375.750000  33534.386719           NaN  0.026064 -0.321212   

                     jet_phi                              jet_E ...   \
subentry         2         0         1         2              0 ...    
entry                                                           ...    
17             NaN -0.017894 -2.452864       NaN  185076.312500 ...    
26             NaN  2.223678 -0.670391       NaN  101834.773438 ...    
43        2.332208       NaN -2.385459 -1.673628            NaN ...    
63             NaN -0.372457  1.453234       NaN  136077.281250 ...    

                         jet_jvt                     jet_MV2c10            \
subentry              2        0         1         2          0         1   
entry                                                                       
17                  NaN  0.99468  0.999721       NaN  -0.798204 -0.660267   
26                  NaN  0.00000  0.000000       NaN  -0.702755 -0.840732   
43        165684.984375      NaN  0.979666  0.997673        NaN -0.942829   
63                  NaN  0.99436  0.843760       NaN  -0.880990 -0.926673   

                  jet_pt_syst            
subentry        2           0    1    2  
entry                                    
17            NaN         0.0  0.0  NaN  
26            NaN         0.0  0.0  NaN  
43       -0.84715         NaN  0.0  0.0  
63            NaN         0.0  0.0  NaN  

[4 rows x 21 columns]

In [27]:
df_jet = df_jet.stack()
df_jet

jet_pt   jet_eta   jet_phi          jet_E   jet_jvt  \
entry subentry                                                               
17    0          40487.335938  2.200428 -0.017894  185076.312500  0.994680   
      1          33965.273438  0.968904 -2.452864   51396.187500  0.999721   
26    0          95023.507812 -0.350044  2.223678  101834.773438  0.000000   
      1          71066.960938 -1.916904 -0.670391  246947.171875  0.000000   
43    1          45823.367188  0.184055 -2.385459   47038.800781  0.979666   
      2          31843.884766  2.332208 -1.673628  165684.984375  0.997673   
63    0         135375.750000  0.026064 -0.372457  136077.281250  0.994360   
      1          33534.386719 -0.321212  1.453234   35426.750000  0.843760   

                jet_MV2c10  jet_pt_syst  
entry subentry                           
17    0          -0.798204          0.0  
      1          -0.660267          0.0  
26    0          -0.702755          0.0  
      1          -0.840732          0.0  
43    1          -0.942829          0.0  
      2          -0.847150          0.0  
63    0          -0.880990          0.0  
      1          -0.926673          0.0

In [28]:
df_jet["subindex"] = df_jet.groupby(level=0).apply(lambda x : x.sort_values("jet_pt", ascending=False)).groupby(level=0).cumcount()
df_jet["subindex"]

entry  subentry
17     0           0
       1           1
26     0           0
       1           1
43     1           0
       2           1
63     0           0
       1           1
Name: subindex, dtype: int64

In [29]:
df_jet = df_jet.set_index("subindex", append=True)
df_jet

jet_pt   jet_eta   jet_phi          jet_E  \
entry subentry subindex                                                     
17    0        0          40487.335938  2.200428 -0.017894  185076.312500   
      1        1          33965.273438  0.968904 -2.452864   51396.187500   
26    0        0          95023.507812 -0.350044  2.223678  101834.773438   
      1        1          71066.960938 -1.916904 -0.670391  246947.171875   
43    1        0          45823.367188  0.184055 -2.385459   47038.800781   
      2        1          31843.884766  2.332208 -1.673628  165684.984375   
63    0        0         135375.750000  0.026064 -0.372457  136077.281250   
      1        1          33534.386719 -0.321212  1.453234   35426.750000   

                          jet_jvt  jet_MV2c10  jet_pt_syst  
entry subentry subindex                                     
17    0        0         0.994680   -0.798204          0.0  
      1        1         0.999721   -0.660267          0.0  
26    0        0         0.000000   -0.702755          0.0  
      1        1         0.000000   -0.840732          0.0  
43    1        0         0.979666   -0.942829          0.0  
      2        1         0.997673   -0.847150          0.0  
63    0        0         0.994360   -0.880990          0.0  
      1        1         0.843760   -0.926673          0.0

In [30]:
df_jet = df_jet.reset_index(level="subentry").drop("subentry", axis=1)
df_jet

jet_pt   jet_eta   jet_phi          jet_E   jet_jvt  \
entry subindex                                                               
17    0          40487.335938  2.200428 -0.017894  185076.312500  0.994680   
      1          33965.273438  0.968904 -2.452864   51396.187500  0.999721   
26    0          95023.507812 -0.350044  2.223678  101834.773438  0.000000   
      1          71066.960938 -1.916904 -0.670391  246947.171875  0.000000   
43    0          45823.367188  0.184055 -2.385459   47038.800781  0.979666   
      1          31843.884766  2.332208 -1.673628  165684.984375  0.997673   
63    0         135375.750000  0.026064 -0.372457  136077.281250  0.994360   
      1          33534.386719 -0.321212  1.453234   35426.750000  0.843760   

                jet_MV2c10  jet_pt_syst  
entry subindex                           
17    0          -0.798204          0.0  
      1          -0.660267          0.0  
26    0          -0.702755          0.0  
      1          -0.840732          0.0  
43    0          -0.942829          0.0  
      1          -0.847150          0.0  
63    0          -0.880990          0.0  
      1          -0.926673          0.0

In [31]:
df_jet = df_jet.unstack(level="subindex")
df_jet

jet_pt                 jet_eta             jet_phi            \
subindex              0             1         0         1         0         1   
entry                                                                           
17         40487.335938  33965.273438  2.200428  0.968904 -0.017894 -2.452864   
26         95023.507812  71066.960938 -0.350044 -1.916904  2.223678 -0.670391   
43         45823.367188  31843.884766  0.184055  2.332208 -2.385459 -1.673628   
63        135375.750000  33534.386719  0.026064 -0.321212 -0.372457  1.453234   

                  jet_E                  jet_jvt           jet_MV2c10  \
subindex              0              1         0         1          0   
entry                                                                   
17        185076.312500   51396.187500  0.994680  0.999721  -0.798204   
26        101834.773438  246947.171875  0.000000  0.000000  -0.702755   
43         47038.800781  165684.984375  0.979666  0.997673  -0.942829   
63        136077.281250   35426.750000  0.994360  0.843760  -0.880990   

                   jet_pt_syst       
subindex         1           0    1  
entry                                
17       -0.660267         0.0  0.0  
26       -0.840732         0.0  0.0  
43       -0.847150         0.0  0.0  
63       -0.926673         0.0  0.0

In [32]:
df_jet.columns = [ col[0] + str(col[1]+1) for col in df_jet.columns.values]
df_jet.columns

Index(['jet_pt1', 'jet_pt2', 'jet_eta1', 'jet_eta2', 'jet_phi1', 'jet_phi2',
       'jet_E1', 'jet_E2', 'jet_jvt1', 'jet_jvt2', 'jet_MV2c101',
       'jet_MV2c102', 'jet_pt_syst1', 'jet_pt_syst2'],
      dtype='object')

In [33]:
df_jet

,jet_pt1,jet_pt2,jet_eta1,jet_eta2,jet_phi1,jet_phi2,jet_E1,jet_E2,jet_jvt1,jet_jvt2,jet_MV2c101,jet_MV2c102,jet_pt_syst1,jet_pt_syst2
entry,,,,,,,,,,,,,,
17,40487.335938,33965.273438,2.200428,0.968904,-0.017894,-2.452864,185076.312500,51396.187500,0.994680,0.999721,-0.798204,-0.660267,0.0,0.0
26,95023.507812,71066.960938,-0.350044,-1.916904,2.223678,-0.670391,101834.773438,246947.171875,0.000000,0.000000,-0.702755,-0.840732,0.0,0.0
43,45823.367188,31843.884766,0.184055,2.332208,-2.385459,-1.673628,47038.800781,165684.984375,0.979666,0.997673,-0.942829,-0.847150,0.0,0.0
63,135375.750000,33534.386719,0.026064,-0.321212,-0.372457,1.453234,136077.281250,35426.750000,0.994360,0.843760,-0.880990,-0.926673,0.0,0.0


In [34]:
from ROOT import TLorentzVector
def invariantMass(l_4vector1, l_4vector2):
    tlv1 = TLorentzVector()
    tlv2 = TLorentzVector()
    tlv1.SetPtEtaPhiE(l_4vector1[0], l_4vector1[1], l_4vector1[2], l_4vector1[3])
    tlv2.SetPtEtaPhiE(l_4vector2[0], l_4vector2[1], l_4vector2[2], l_4vector2[3])
    return (tlv1+tlv2).M()

Welcome to JupyROOT 6.16/00


In [35]:
df_jet['mjj'] = df_jet.apply(lambda x: invariantMass([x.jet_pt1, x.jet_eta1, x.jet_phi1, x.jet_E1], [x.jet_pt2, x.jet_eta2, x.jet_phi2, x.jet_E2]), axis=1)
df_jet

,jet_pt1,jet_pt2,jet_eta1,jet_eta2,jet_phi1,jet_phi2,jet_E1,jet_E2,jet_jvt1,jet_jvt2,jet_MV2c101,jet_MV2c102,jet_pt_syst1,jet_pt_syst2,mjj
entry,,,,,,,,,,,,,,,
17,40487.335938,33965.273438,2.200428,0.968904,-0.017894,-2.452864,185076.312500,51396.187500,0.994680,0.999721,-0.798204,-0.660267,0.0,0.0,85597.133521
26,95023.507812,71066.960938,-0.350044,-1.916904,2.223678,-0.670391,101834.773438,246947.171875,0.000000,0.000000,-0.702755,-0.840732,0.0,0.0,218123.114926
43,45823.367188,31843.884766,0.184055,2.332208,-2.385459,-1.673628,47038.800781,165684.984375,0.979666,0.997673,-0.942829,-0.847150,0.0,0.0,103472.329165
63,135375.750000,33534.386719,0.026064,-0.321212,-0.372457,1.453234,136077.281250,35426.750000,0.994360,0.843760,-0.880990,-0.926673,0.0,0.0,110438.321235


In [36]:
df_flat = pd.concat([df_flat, df_jet], axis=1, sort=False)
df_flat

,runNumber,eventNumber,channelNumber,mcWeight,scaleFactor_PILEUP,scaleFactor_ELE,scaleFactor_MUON,scaleFactor_PHOTON,scaleFactor_TAU,scaleFactor_BTAG,...,jet_phi2,jet_E1,jet_E2,jet_jvt1,jet_jvt2,jet_MV2c101,jet_MV2c102,jet_pt_syst1,jet_pt_syst2,mjj
entry,,,,,,,,,,,,,,,,,,,,,
0,300279,109293365,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,300279,109389491,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,300279,109416872,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,300279,109473292,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,300279,109509709,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,300279,109715295,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,300279,111328379,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,300279,111256237,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,300279,110239756,300279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
df_lep = tree.pandas.df(branches="/lep_(?!tru).*/i", entrystart=None, entrystop=100, flatten=True)
df_lep = df_lep.loc[:,"lep_pt":]
df_lep

lep_pt   lep_eta   lep_phi          lep_E      lep_z0  \
entry subentry                                                                 
0     0          33144.898438  2.176089  0.214251  147913.468750    0.041109   
      1          21067.384766 -0.139190 -2.670418   21271.792969   -0.030600   
1     0          42871.246094  0.067116 -0.132653   42967.839844   -0.030441   
      1          37205.386719 -0.946074 -2.926308   55135.394531    0.060783   
2     0          33070.187500 -0.366347  2.462502   35314.296875   -0.082481   
      1          29222.083984 -2.146044 -0.861610  126647.132812    0.099033   
3     0          28597.910156  1.951096 -1.353901  102645.257812    0.000303   
      1          19120.683594  2.089655  1.913332   78450.765625    0.094156   
4     0          49459.628906 -1.319296  1.621212   99119.835938   -0.062122   
      1          35819.855469 -0.482460 -1.404498   40070.199219   -0.042025   
5     0          63139.535156 -0.228282  0.157530   64791.960938    0.116914   
      1          18481.808594 -1.915037 -2.493073   64081.375000   -0.032120   
6     0          49670.843750  1.559802  1.389487  123383.804688   -0.227428   
      1          36633.945312  0.832217 -1.670374   50069.304688    0.041323   
7     0          27137.146484 -2.158771 -3.078488  119077.203125    0.136036   
      1          25529.798828  0.055718  0.107797   25569.656250   -0.019691   
8     0          50368.695312  0.262065 -1.761118   52108.335938   -0.016087   
      1          46622.074219  2.330840  2.900253  242057.031250    0.168331   
9     0          34681.558594  0.149678  2.475889   35070.937500   -0.004812   
      1          30784.708984 -0.992247 -0.792478   47224.359375    0.030548   
10    0          35948.933594 -2.050641 -2.323505  142025.953125   -0.058674   
      1          21704.658203  0.067306 -0.392930   21754.095703   -0.100178   
11    0          46261.167969 -0.457101 -1.771979   51178.957031    0.018188   
      1          45946.367188 -0.034544  1.797653   45973.906250   -0.033455   
12    0          35353.191406  0.905338 -2.229646   50858.609375   -0.031647   
      1          30858.828125 -0.803876  0.455485   41378.203125    0.017056   
13    0          43318.074219 -0.722018  1.876161   55108.269531   -0.016918   
      1          37700.683594  0.034469 -1.356414   37723.082031   -0.012112   
14    0          40537.402344  0.628942  0.066884   48822.972656    0.006360   
      1          44665.652344  1.170512 -2.989091   78920.960938   -0.004982   
...                       ...       ...       ...            ...         ...   
85    0          25742.578125 -1.730082 -0.877124   74890.187500    0.002708   
      1         330961.937500  0.053169 -1.340519  331429.875000  181.777496   
86    0          75585.937500  2.038188  0.319473  295047.781250   -0.078896   
      1          26402.333984  1.850575 -2.550588   86079.843750   -0.153698   
87    0          60810.488281  0.180575  2.846802   61804.710938    0.051751   
      1          16223.000977  0.908059 -0.474188   23383.921875    0.012172   
88    0          38256.851562  1.573181 -2.530406   96203.460938   -0.117786   
      1          14727.200195  0.927234 -0.015514   21524.939453    0.040760   
89    0          41769.542969  0.422835 -0.919026   45559.609375    0.099171   
      1          45586.363281 -0.092476  2.227000   45781.546875    0.001151   
90    0          50171.257812 -1.743628 -1.137076  147827.937500   -0.044762   
      1          42482.636719 -0.801089  1.753288   56858.898438   -0.035691   
91    0          44344.125000 -0.930254  1.764987   64955.433594    0.029722   
      1          36725.933594 -1.831367 -1.475564  117570.851562    0.077971   
92    0          32555.955078 -0.219534 -1.427429   33343.628906    0.157327   
      1          24904.498047 -0.409213  1.911747   27018.955078    0.060693   
93    0          55504.910156  0.231176 -1.833976   56994.777344    0.145558   
      1           9344.

In [56]:
l_cuts_lep = ["lep_pt > 25e3",
              "((lep_type == 11) & (abs(lep_eta) < 2.47)) | ((lep_type == 13) & (abs(lep_eta) < 2.4))"]
df_lep = df_lep.query(" & ".join(l_cuts_lep))
df_lep

lep_pt   lep_eta   lep_phi          lep_E      lep_z0  \
entry subentry                                                                 
0     0          33144.898438  2.176089  0.214251  147913.468750    0.041109   
1     0          42871.246094  0.067116 -0.132653   42967.839844   -0.030441   
      1          37205.386719 -0.946074 -2.926308   55135.394531    0.060783   
2     0          33070.187500 -0.366347  2.462502   35314.296875   -0.082481   
      1          29222.083984 -2.146044 -0.861610  126647.132812    0.099033   
3     0          28597.910156  1.951096 -1.353901  102645.257812    0.000303   
4     0          49459.628906 -1.319296  1.621212   99119.835938   -0.062122   
      1          35819.855469 -0.482460 -1.404498   40070.199219   -0.042025   
5     0          63139.535156 -0.228282  0.157530   64791.960938    0.116914   
      1          18481.808594 -1.915037 -2.493073   64081.375000   -0.032120   
6     0          49670.843750  1.559802  1.389487  123383.804688   -0.227428   
      1          36633.945312  0.832217 -1.670374   50069.304688    0.041323   
7     0          27137.146484 -2.158771 -3.078488  119077.203125    0.136036   
      1          25529.798828  0.055718  0.107797   25569.656250   -0.019691   
8     0          50368.695312  0.262065 -1.761118   52108.335938   -0.016087   
      1          46622.074219  2.330840  2.900253  242057.031250    0.168331   
9     0          34681.558594  0.149678  2.475889   35070.937500   -0.004812   
      1          30784.708984 -0.992247 -0.792478   47224.359375    0.030548   
10    0          35948.933594 -2.050641 -2.323505  142025.953125   -0.058674   
      1          21704.658203  0.067306 -0.392930   21754.095703   -0.100178   
11    0          46261.167969 -0.457101 -1.771979   51178.957031    0.018188   
      1          45946.367188 -0.034544  1.797653   45973.906250   -0.033455   
12    0          35353.191406  0.905338 -2.229646   50858.609375   -0.031647   
      1          30858.828125 -0.803876  0.455485   41378.203125    0.017056   
13    0          43318.074219 -0.722018  1.876161   55108.269531   -0.016918   
      1          37700.683594  0.034469 -1.356414   37723.082031   -0.012112   
14    0          40537.402344  0.628942  0.066884   48822.972656    0.006360   
      1          44665.652344  1.170512 -2.989091   78920.960938   -0.004982   
15    0          41374.218750  0.253816  0.332898   42714.105469    0.004787   
      1          42279.253906  1.249309 -2.838921   79794.375000   -0.087181   
...                       ...       ...       ...            ...         ...   
83    0          52966.238281  1.716061 -0.364654  152075.375000   -0.005005   
      1           7746.626953  1.359759  0.337046   16082.253906    0.063595   
84    0          36169.085938  2.130260  2.796939  154367.265625    0.219857   
      1          48794.535156  1.483048 -0.577358  113039.937500   -0.018294   
85    0          25742.578125 -1.730082 -0.877124   74890.187500    0.002708   
      1         330961.937500  0.053169 -1.340519  331429.875000  181.777496   
86    0          75585.937500  2.038188  0.319473  295047.781250   -0.078896   
      1          26402.333984  1.850575 -2.550588   86079.843750   -0.153698   
87    0          60810.488281  0.180575  2.846802   61804.710938    0.051751   
88    0          38256.851562  1.573181 -2.530406   96203.460938   -0.117786   
89    0          41769.542969  0.422835 -0.919026   45559.609375    0.099171   
      1          45586.363281 -0.092476  2.227000   45781.546875    0.001151   
90    0          50171.257812 -1.743628 -1.137076  147827.937500   -0.044762   
      1          42482.636719 -0.801089  1.753288   56858.898438   -0.035691   
91    0          44344.125000 -0.930254  1.764987   64955.433594    0.029722   
      1          36725.933594 -1.831367 -1.475564  117570.851562    0.077971   
92    0          32555.955078 -0.219534 -1.427429   33343.628906    0.157327   
93    0          55504.

In [58]:
df_lep = df_lep.unstack()
df_lep

lep_pt                  lep_eta             lep_phi  \
subentry              0              1         0         1         0   
entry                                                                  
0          33144.898438            NaN  2.176089       NaN  0.214251   
1          42871.246094   37205.386719  0.067116 -0.946074 -0.132653   
2          33070.187500   29222.083984 -0.366347 -2.146044  2.462502   
3          28597.910156            NaN  1.951096       NaN -1.353901   
4          49459.628906   35819.855469 -1.319296 -0.482460  1.621212   
5          63139.535156   18481.808594 -0.228282 -1.915037  0.157530   
6          49670.843750   36633.945312  1.559802  0.832217  1.389487   
7          27137.146484   25529.798828 -2.158771  0.055718 -3.078488   
8          50368.695312   46622.074219  0.262065  2.330840 -1.761118   
9          34681.558594   30784.708984  0.149678 -0.992247  2.475889   
10         35948.933594   21704.658203 -2.050641  0.067306 -2.323505   
11         46261.167969   45946.367188 -0.457101 -0.034544 -1.771979   
12         35353.191406   30858.828125  0.905338 -0.803876 -2.229646   
13         43318.074219   37700.683594 -0.722018  0.034469  1.876161   
14         40537.402344   44665.652344  0.628942  1.170512  0.066884   
15         41374.218750   42279.253906  0.253816  1.249309  0.332898   
16         55903.132812   27114.072266  1.377304 -1.801778  2.649427   
17         26444.722656   10483.013672  2.266445  1.935604  2.395073   
18         29561.855469   26253.234375 -1.717983  0.299206  2.884985   
19         34499.980469   29882.421875  0.662654 -0.658610  0.514207   
20        109138.304688   39743.039062 -0.932118 -2.033489 -2.524518   
21         32566.789062   22635.314453 -0.440597  1.852220  2.623239   
22         41859.957031   51860.132812  1.377881  1.577967 -2.425950   
23         52863.117188            NaN -1.361719       NaN  0.189015   
24         31277.853516   21439.810547  1.943629 -0.058927 -2.391421   
25         37826.039062   36743.019531 -0.883461 -1.062985 -2.765679   
26         49835.671875   45014.269531 -0.190989 -0.028876 -1.782506   
27         43601.722656   25592.119141  1.738111  0.067109 -1.635545   
28         28408.740234            NaN  1.203529       NaN -0.499178   
29         44390.398438   39350.449219 -1.356906 -1.963253 -1.609735   
...                 ...            ...       ...       ...       ...   
70         38626.796875    8235.175781  1.547385  1.049640 -2.042007   
71         44910.027344   44031.250000  1.544697  1.556988  2.207810   
72         61141.085938   26436.773438 -2.155302 -1.401769  1.817581   
73         25010.275391   18335.119141  1.965301  1.703894  2.375652   
74         42880.132812   49101.148438  1.983247  1.682675  0.987586   
75         43659.382812   43578.066406 -1.025836 -1.501633 -1.607477   
76         44637.902344   40786.160156 -0.149012 -0.844010 -3.095932   
77         68448.382812            NaN -2.347627       NaN  0.360799   
78         35384.199219   28507.283203  1.584595 -0.010002  1.260056   
79         33184.792969   39002.671875  0.471212 -1.084828  2.380660   
80         49186.359375   37137.976562 -1.031121 -1.339616 -1.556749   
81        107463.429688   39151.929688 -0.504153 -1.195364  2.792781   
82         30368.224609   25987.732422  1.821124 -0.274072 -1.548637   
83         52966.238281    7746.626953  1.716061  1.359759 -0.364654   
84         36169.085938   48794.535156  2.130260  1.483048  2.796939   
85         25742.578125  330961.937500 -1.730082  0.053169 -0.877124   
86         75585.937500   26402.333984  2.038188  1.850575  0.319473   
87         60810.488281            NaN  0.180575       NaN  2.846802   
88         38256.851562            NaN  1.573181       NaN -2.530406   
89         41769.542969   45586.363281  0.422835 -0.092476 -0.919026   
90         50171.257812   42482.636719 -1.743628 -0.801089 -1.137076   
91         44344.125000   36725.933594 -0.930254 -1.83136

In [59]:
df_lep = df_lep[n_lep_sig == 2]
df_lep

lep_pt                  lep_eta             lep_phi  \
subentry              0              1         0         1         0   
entry                                                                  
1          42871.246094   37205.386719  0.067116 -0.946074 -0.132653   
2          33070.187500   29222.083984 -0.366347 -2.146044  2.462502   
4          49459.628906   35819.855469 -1.319296 -0.482460  1.621212   
5          63139.535156   18481.808594 -0.228282 -1.915037  0.157530   
6          49670.843750   36633.945312  1.559802  0.832217  1.389487   
7          27137.146484   25529.798828 -2.158771  0.055718 -3.078488   
8          50368.695312   46622.074219  0.262065  2.330840 -1.761118   
9          34681.558594   30784.708984  0.149678 -0.992247  2.475889   
10         35948.933594   21704.658203 -2.050641  0.067306 -2.323505   
11         46261.167969   45946.367188 -0.457101 -0.034544 -1.771979   
12         35353.191406   30858.828125  0.905338 -0.803876 -2.229646   
13         43318.074219   37700.683594 -0.722018  0.034469  1.876161   
14         40537.402344   44665.652344  0.628942  1.170512  0.066884   
15         41374.218750   42279.253906  0.253816  1.249309  0.332898   
16         55903.132812   27114.072266  1.377304 -1.801778  2.649427   
17         26444.722656   10483.013672  2.266445  1.935604  2.395073   
18         29561.855469   26253.234375 -1.717983  0.299206  2.884985   
19         34499.980469   29882.421875  0.662654 -0.658610  0.514207   
20        109138.304688   39743.039062 -0.932118 -2.033489 -2.524518   
21         32566.789062   22635.314453 -0.440597  1.852220  2.623239   
22         41859.957031   51860.132812  1.377881  1.577967 -2.425950   
24         31277.853516   21439.810547  1.943629 -0.058927 -2.391421   
25         37826.039062   36743.019531 -0.883461 -1.062985 -2.765679   
26         49835.671875   45014.269531 -0.190989 -0.028876 -1.782506   
27         43601.722656   25592.119141  1.738111  0.067109 -1.635545   
29         44390.398438   39350.449219 -1.356906 -1.963253 -1.609735   
30         42287.117188   27343.341797  2.096896  1.352833 -1.161422   
31         41354.621094   43679.871094  1.090576  0.349798 -0.842754   
32         30538.962891   34688.828125 -0.096403 -1.083643 -0.765579   
34         45738.093750   36554.648438  0.433773 -0.514621 -1.869180   
...                 ...            ...       ...       ...       ...   
64         38668.269531   40893.511719  2.037512  1.197688  2.497566   
66         42604.375000   39241.308594  0.860947  1.952806 -2.786951   
67         50523.785156   30253.580078  1.951069  2.212003 -2.880038   
68         49506.484375   43226.398438 -0.215141 -0.147518  0.828979   
69         25880.279297   15362.814453  0.706640  1.410422 -2.391230   
70         38626.796875    8235.175781  1.547385  1.049640 -2.042007   
71         44910.027344   44031.250000  1.544697  1.556988  2.207810   
72         61141.085938   26436.773438 -2.155302 -1.401769  1.817581   
73         25010.275391   18335.119141  1.965301  1.703894  2.375652   
74         42880.132812   49101.148438  1.983247  1.682675  0.987586   
75         43659.382812   43578.066406 -1.025836 -1.501633 -1.607477   
76         44637.902344   40786.160156 -0.149012 -0.844010 -3.095932   
78         35384.199219   28507.283203  1.584595 -0.010002  1.260056   
79         33184.792969   39002.671875  0.471212 -1.084828  2.380660   
80         49186.359375   37137.976562 -1.031121 -1.339616 -1.556749   
81        107463.429688   39151.929688 -0.504153 -1.195364  2.792781   
82         30368.224609   25987.732422  1.821124 -0.274072 -1.548637   
83         52966.238281    7746.626953  1.716061  1.359759 -0.364654   
84         36169.085938   48794.535156  2.130260  1.483048  2.796939   
85         25742.578125  330961.937500 -1.730082  0.053169 -0.877124   
86         75585.937500   26402.333984  2.038188  1.850575  0.319473   
89         41769.542969   45586.363281  0.422835 -0.09247

In [60]:
df_lep = df_lep.stack()
df_lep

lep_pt   lep_eta   lep_phi          lep_E      lep_z0  \
entry subentry                                                                 
1     0          42871.246094  0.067116 -0.132653   42967.839844   -0.030441   
      1          37205.386719 -0.946074 -2.926308   55135.394531    0.060783   
2     0          33070.187500 -0.366347  2.462502   35314.296875   -0.082481   
      1          29222.083984 -2.146044 -0.861610  126647.132812    0.099033   
4     0          49459.628906 -1.319296  1.621212   99119.835938   -0.062122   
      1          35819.855469 -0.482460 -1.404498   40070.199219   -0.042025   
5     0          63139.535156 -0.228282  0.157530   64791.960938    0.116914   
      1          18481.808594 -1.915037 -2.493073   64081.375000   -0.032120   
6     0          49670.843750  1.559802  1.389487  123383.804688   -0.227428   
      1          36633.945312  0.832217 -1.670374   50069.304688    0.041323   
7     0          27137.146484 -2.158771 -3.078488  119077.203125    0.136036   
      1          25529.798828  0.055718  0.107797   25569.656250   -0.019691   
8     0          50368.695312  0.262065 -1.761118   52108.335938   -0.016087   
      1          46622.074219  2.330840  2.900253  242057.031250    0.168331   
9     0          34681.558594  0.149678  2.475889   35070.937500   -0.004812   
      1          30784.708984 -0.992247 -0.792478   47224.359375    0.030548   
10    0          35948.933594 -2.050641 -2.323505  142025.953125   -0.058674   
      1          21704.658203  0.067306 -0.392930   21754.095703   -0.100178   
11    0          46261.167969 -0.457101 -1.771979   51178.957031    0.018188   
      1          45946.367188 -0.034544  1.797653   45973.906250   -0.033455   
12    0          35353.191406  0.905338 -2.229646   50858.609375   -0.031647   
      1          30858.828125 -0.803876  0.455485   41378.203125    0.017056   
13    0          43318.074219 -0.722018  1.876161   55108.269531   -0.016918   
      1          37700.683594  0.034469 -1.356414   37723.082031   -0.012112   
14    0          40537.402344  0.628942  0.066884   48822.972656    0.006360   
      1          44665.652344  1.170512 -2.989091   78920.960938   -0.004982   
15    0          41374.218750  0.253816  0.332898   42714.105469    0.004787   
      1          42279.253906  1.249309 -2.838921   79794.375000   -0.087181   
16    0          55903.132812  1.377304  2.649427  117856.679688    0.091293   
      1          27114.072266 -1.801778 -1.098997   84398.289062   -0.043148   
...                       ...       ...       ...            ...         ...   
81    0         107463.429688 -0.504153  2.792781  121412.187500    0.056360   
      1          39151.929688 -1.195364 -2.132873   70617.429688    0.018229   
82    0          30368.224609  1.821124 -1.548637   96277.023438   -0.367655   
      1          25987.732422 -0.274072  2.550966   26969.894531    0.008793   
83    0          52966.238281  1.716061 -0.364654  152075.375000   -0.005005   
      1           7746.626953  1.359759  0.337046   16082.253906    0.063595   
84    0          36169.085938  2.130260  2.796939  154367.265625    0.219857   
      1          48794.535156  1.483048 -0.577358  113039.937500   -0.018294   
85    0          25742.578125 -1.730082 -0.877124   74890.187500    0.002708   
      1         330961.937500  0.053169 -1.340519  331429.875000  181.777496   
86    0          75585.937500  2.038188  0.319473  295047.781250   -0.078896   
      1          26402.333984  1.850575 -2.550588   86079.843750   -0.153698   
89    0          41769.542969  0.422835 -0.919026   45559.609375    0.099171   
      1          45586.363281 -0.092476  2.227000   45781.546875    0.001151   
90    0          50171.257812 -1.743628 -1.137076  147827.937500   -0.044762   
      1          42482.636719 -0.801089  1.753288   56858.898438   -0.035691   
91    0          44344.125000 -0.930254  1.764987   64955.433594    0.029722   
      1          36725.

In [61]:
df_lep_sorted = df_lep.groupby(level=0).apply(lambda x : x.sort_values("lep_pt", ascending=False))
df_lep_sorted = df_lep_sorted.reset_index(level=0).drop("entry", axis=1)
df_lep["subindex"] = df_lep_sorted.groupby(level=0).cumcount()
df_lep = df_lep.set_index("subindex", append=True)
df_lep = df_lep.reset_index(level="subentry").drop("subentry", axis=1)
df_lep = df_lep.unstack(level="subindex")
df_lep

lep_pt                 lep_eta             lep_phi            \
subindex              0             1         0         1         0         1   
entry                                                                           
1          42871.246094  37205.386719  0.067116 -0.946074 -0.132653 -2.926308   
2          33070.187500  29222.083984 -0.366347 -2.146044  2.462502 -0.861610   
4          49459.628906  35819.855469 -1.319296 -0.482460  1.621212 -1.404498   
5          63139.535156  18481.808594 -0.228282 -1.915037  0.157530 -2.493073   
6          49670.843750  36633.945312  1.559802  0.832217  1.389487 -1.670374   
7          27137.146484  25529.798828 -2.158771  0.055718 -3.078488  0.107797   
8          50368.695312  46622.074219  0.262065  2.330840 -1.761118  2.900253   
9          34681.558594  30784.708984  0.149678 -0.992247  2.475889 -0.792478   
10         35948.933594  21704.658203 -2.050641  0.067306 -2.323505 -0.392930   
11         46261.167969  45946.367188 -0.457101 -0.034544 -1.771979  1.797653   
12         35353.191406  30858.828125  0.905338 -0.803876 -2.229646  0.455485   
13         43318.074219  37700.683594 -0.722018  0.034469  1.876161 -1.356414   
14         44665.652344  40537.402344  1.170512  0.628942 -2.989091  0.066884   
15         42279.253906  41374.218750  1.249309  0.253816 -2.838921  0.332898   
16         55903.132812  27114.072266  1.377304 -1.801778  2.649427 -1.098997   
17         26444.722656  10483.013672  2.266445  1.935604  2.395073 -1.325978   
18         29561.855469  26253.234375 -1.717983  0.299206  2.884985 -0.320330   
19         34499.980469  29882.421875  0.662654 -0.658610  0.514207 -2.792500   
20        109138.304688  39743.039062 -0.932118 -2.033489 -2.524518 -0.349218   
21         32566.789062  22635.314453 -0.440597  1.852220  2.623239 -0.460026   
22         51860.132812  41859.957031  1.577967  1.377881  0.982246 -2.425950   
24         31277.853516  21439.810547  1.943629 -0.058927 -2.391421  0.944848   
25         37826.039062  36743.019531 -0.883461 -1.062985 -2.765679  0.714493   
26         49835.671875  45014.269531 -0.190989 -0.028876 -1.782506  0.871316   
27         43601.722656  25592.119141  1.738111  0.067109 -1.635545  1.017639   
29         44390.398438  39350.449219 -1.356906 -1.963253 -1.609735  1.403175   
30         42287.117188  27343.341797  2.096896  1.352833 -1.161422  1.632338   
31         43679.871094  41354.621094  0.349798  1.090576  2.614278 -0.842754   
32         34688.828125  30538.962891 -1.083643 -0.096403  2.445173 -0.765579   
34         45738.093750  36554.648438  0.433773 -0.514621 -1.869180  1.193118   
...                 ...           ...       ...       ...       ...       ...   
64         40893.511719  38668.269531  1.197688  2.037512 -0.675512  2.497566   
66         42604.375000  39241.308594  0.860947  1.952806 -2.786951  1.018207   
67         50523.785156  30253.580078  1.951069  2.212003 -2.880038 -0.010321   
68         49506.484375  43226.398438 -0.215141 -0.147518  0.828979 -2.218365   
69         25880.279297  15362.814453  0.706640  1.410422 -2.391230  0.683047   
70         38626.796875   8235.175781  1.547385  1.049640 -2.042007 -1.735511   
71         44910.027344  44031.250000  1.544697  1.556988  2.207810 -0.905965   
72         61141.085938  26436.773438 -2.155302 -1.401769  1.817581 -1.527705   
73         25010.275391  18335.119141  1.965301  1.703894  2.375652 -0.837416   
74         49101.148438  42880.132812  1.682675  1.983247 -2.286255  0.987586   
75         43659.382812  43578.066406 -1.025836 -1.501633 -1.607477  1.578252   
76         44637.902344  40786.160156 -0.149012 -0.844010 -3.095932 -0.092380   
78         35384.199219  28507.283203  1.584595 -0.010002  1.260056 -1.588112   
79         39002.671875  33184.792969 -1.084828  0.471212 -1.514102  2.380660   
80         49186.359375  37137.976562 -1.031121 -1.339616 -1.556749  1.634659   
81        107463.429688  39151.929688 -0.504153 

In [62]:
df_lep.columns = [col[0]+str(col[1]+1) for col in df_lep.columns.values]
df_lep.columns

Index(['lep_pt1', 'lep_pt2', 'lep_eta1', 'lep_eta2', 'lep_phi1', 'lep_phi2',
       'lep_E1', 'lep_E2', 'lep_z01', 'lep_z02', 'lep_charge1', 'lep_charge2',
       'lep_type1', 'lep_type2', 'lep_isTightID1', 'lep_isTightID2',
       'lep_ptcone301', 'lep_ptcone302', 'lep_etcone201', 'lep_etcone202',
       'lep_trackd0pvunbiased1', 'lep_trackd0pvunbiased2',
       'lep_tracksigd0pvunbiased1', 'lep_tracksigd0pvunbiased2',
       'lep_pt_syst1', 'lep_pt_syst2'],
      dtype='object')

In [63]:
df_lep

,lep_pt1,lep_pt2,lep_eta1,lep_eta2,lep_phi1,lep_phi2,lep_E1,lep_E2,lep_z01,lep_z02,...,lep_ptcone301,lep_ptcone302,lep_etcone201,lep_etcone202,lep_trackd0pvunbiased1,lep_trackd0pvunbiased2,lep_tracksigd0pvunbiased1,lep_tracksigd0pvunbiased2,lep_pt_syst1,lep_pt_syst2
entry,,,,,,,,,,,,,,,,,,,,,
1,42871.246094,37205.386719,0.067116,-0.946074,-0.132653,-2.926308,42967.839844,55135.394531,-0.030441,0.060783,...,3140.769287,2661.424805,-1359.720703,-101.417023,0.020707,-0.023976,1.708643,1.298120,0.0,0.0
2,33070.187500,29222.083984,-0.366347,-2.146044,2.462502,-0.861610,35314.296875,126647.132812,-0.082481,0.099033,...,0.000000,0.000000,-343.630188,293.672791,-0.002222,0.149375,0.248513,2.049124,0.0,0.0
4,49459.628906,35819.855469,-1.319296,-0.482460,1.621212,-1.404498,99119.835938,40070.199219,-0.062122,-0.042025,...,0.000000,0.000000,-514.342468,201.623901,0.018966,-0.003544,1.575740,0.375530,0.0,0.0
5,63139.535156,18481.808594,-0.228282,-1.915037,0.157530,-2.493073,64791.960938,64081.375000,0.116914,-0.032120,...,0.000000,0.000000,438.852600,1401.662720,0.009244,0.005339,1.757471,0.380774,0.0,0.0
6,49670.843750,36633.945312,1.559802,0.832217,1.389487,-1.670374,123383.804688,50069.304688,-0.227428,0.041323,...,0.000000,0.000000,-511.167053,146.605042,0.013305,0.016785,1.368054,2.117139,0.0,0.0
7,27137.146484,25529.798828,-2.158771,0.055718,-3.078488,0.107797,119077.203125,25569.656250,0.136036,-0.019691,...,0.000000,0.000000,-544.768372,-201.065674,0.029119,0.005794,2.782785,0.609303,0.0,0.0
8,50368.695312,46622.074219,0.262065,2.330840,-1.761118,2.900253,52108.335938,242057.031250,-0.016087,0.168331,...,1414.020874,0.000000,50.798492,-120.564590,0.035539,-0.007626,5.180129,0.793478,0.0,0.0
9,34681.558594,30784.708984,0.149678,-0.992247,2.475889,-0.792478,35070.937500,47224.359375,-0.004812,0.030548,...,0.000000,0.000000,1845.624268,-176.554993,-0.003620,0.000955,0.395175,0.133722,0.0,0.0
10,35948.933594,21704.658203,-2.050641,0.067306,-2.323505,-0.392930,142025.953125,21754.095703,-0.058674,-0.100178,...,0.000000,0.000000,796.835754,-433.422577,0.005617,0.039837,0.544657,3.503185,0.0,0.0


In [64]:
l_cuts_SFOS = ["lep_type1 == lep_type2", "lep_charge1 != lep_charge2"]
df_lep = df_lep.query("&".join(l_cuts_SFOS))
df_lep

,lep_pt1,lep_pt2,lep_eta1,lep_eta2,lep_phi1,lep_phi2,lep_E1,lep_E2,lep_z01,lep_z02,...,lep_ptcone301,lep_ptcone302,lep_etcone201,lep_etcone202,lep_trackd0pvunbiased1,lep_trackd0pvunbiased2,lep_tracksigd0pvunbiased1,lep_tracksigd0pvunbiased2,lep_pt_syst1,lep_pt_syst2
entry,,,,,,,,,,,,,,,,,,,,,
1,42871.246094,37205.386719,0.067116,-0.946074,-0.132653,-2.926308,42967.839844,55135.394531,-0.030441,0.060783,...,3140.769287,2661.424805,-1359.720703,-101.417023,0.020707,-0.023976,1.708643,1.298120,0.0,0.0
2,33070.187500,29222.083984,-0.366347,-2.146044,2.462502,-0.861610,35314.296875,126647.132812,-0.082481,0.099033,...,0.000000,0.000000,-343.630188,293.672791,-0.002222,0.149375,0.248513,2.049124,0.0,0.0
4,49459.628906,35819.855469,-1.319296,-0.482460,1.621212,-1.404498,99119.835938,40070.199219,-0.062122,-0.042025,...,0.000000,0.000000,-514.342468,201.623901,0.018966,-0.003544,1.575740,0.375530,0.0,0.0
5,63139.535156,18481.808594,-0.228282,-1.915037,0.157530,-2.493073,64791.960938,64081.375000,0.116914,-0.032120,...,0.000000,0.000000,438.852600,1401.662720,0.009244,0.005339,1.757471,0.380774,0.0,0.0
6,49670.843750,36633.945312,1.559802,0.832217,1.389487,-1.670374,123383.804688,50069.304688,-0.227428,0.041323,...,0.000000,0.000000,-511.167053,146.605042,0.013305,0.016785,1.368054,2.117139,0.0,0.0
7,27137.146484,25529.798828,-2.158771,0.055718,-3.078488,0.107797,119077.203125,25569.656250,0.136036,-0.019691,...,0.000000,0.000000,-544.768372,-201.065674,0.029119,0.005794,2.782785,0.609303,0.0,0.0
8,50368.695312,46622.074219,0.262065,2.330840,-1.761118,2.900253,52108.335938,242057.031250,-0.016087,0.168331,...,1414.020874,0.000000,50.798492,-120.564590,0.035539,-0.007626,5.180129,0.793478,0.0,0.0
9,34681.558594,30784.708984,0.149678,-0.992247,2.475889,-0.792478,35070.937500,47224.359375,-0.004812,0.030548,...,0.000000,0.000000,1845.624268,-176.554993,-0.003620,0.000955,0.395175,0.133722,0.0,0.0
10,35948.933594,21704.658203,-2.050641,0.067306,-2.323505,-0.392930,142025.953125,21754.095703,-0.058674,-0.100178,...,0.000000,0.000000,796.835754,-433.422577,0.005617,0.039837,0.544657,3.503185,0.0,0.0


In [65]:
df_flat = pd.concat([df_flat, df_lep], axis=1, sort=False)
df_flat

,runNumber,eventNumber,channelNumber,mcWeight,scaleFactor_PILEUP,scaleFactor_ELE,scaleFactor_MUON,scaleFactor_PHOTON,scaleFactor_TAU,scaleFactor_BTAG,...,lep_ptcone301,lep_ptcone302,lep_etcone201,lep_etcone202,lep_trackd0pvunbiased1,lep_trackd0pvunbiased2,lep_tracksigd0pvunbiased1,lep_tracksigd0pvunbiased2,lep_pt_syst1,lep_pt_syst2
entry,,,,,,,,,,,,,,,,,,,,,
0,300279.0,109293365.0,300279.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,300279.0,109389491.0,300279.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3140.769287,2661.424805,-1359.720703,-101.417023,0.020707,-0.023976,1.708643,1.298120,0.0,0.0
2,300279.0,109416872.0,300279.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,-343.630188,293.672791,-0.002222,0.149375,0.248513,2.049124,0.0,0.0
3,300279.0,109473292.0,300279.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,300279.0,109509709.0,300279.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,-514.342468,201.623901,0.018966,-0.003544,1.575740,0.375530,0.0,0.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,438.852600,1401.662720,0.009244,0.005339,1.757471,0.380774,0.0,0.0
6,300279.0,109715295.0,300279.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,-511.167053,146.605042,0.013305,0.016785,1.368054,2.117139,0.0,0.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,-544.768372,-201.065674,0.029119,0.005794,2.782785,0.609303,0.0,0.0
8,300279.0,111328379.0,300279.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1414.020874,0.000000,50.798492,-120.564590,0.035539,-0.007626,5.180129,0.793478,0.0,0.0


In [66]:
df_flat.dropna(axis='index', inplace=True)
df_flat

,runNumber,eventNumber,channelNumber,mcWeight,scaleFactor_PILEUP,scaleFactor_ELE,scaleFactor_MUON,scaleFactor_PHOTON,scaleFactor_TAU,scaleFactor_BTAG,...,lep_ptcone301,lep_ptcone302,lep_etcone201,lep_etcone202,lep_trackd0pvunbiased1,lep_trackd0pvunbiased2,lep_tracksigd0pvunbiased1,lep_tracksigd0pvunbiased2,lep_pt_syst1,lep_pt_syst2
entry,,,,,,,,,,,,,,,,,,,,,
17,300279.0,110560487.0,300279.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1066.92041,-300.489868,-598.937500,0.007838,0.018884,0.698877,0.727122,0.0,0.0
26,300279.0,112328350.0,300279.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,901.817261,1549.864502,0.010420,0.007699,1.591577,0.535478,0.0,0.0
63,300279.0,110965616.0,300279.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,231.532928,-386.844727,0.019275,0.000916,2.126153,0.067409,0.0,0.0


In [56]:
df_flat.values[0]*1

array([284500.0, 130.0, 392302.0, 1.0, 0.18753056228160858, 1.0,
       0.9967801570892334, 1.0, 1.0, 0.9838106632232666, 0.96222323179245,
       1.0, 1.0, 1.0, 0, 1, 0, 0, 0, 2.0, 333983.8125, -0.482479065656662,
       2.0, 0.0, 0.0, 0.0, 0.0, 4881.03125, 142721.78125, 114061.53125,
       0.4014224410057068, 0.5082284808158875, 2.95322322845459,
       1.47542142868042, 154810.90625, 129579.2734375, 0.9811878800392151,
       0.9932543039321899, 0.0, 4.0, 0, 1, -0.9357396960258484,
       0.4180818796157837, 3567.779052734375, 2506.335205078125, 1, 1, 1,
       0, 164413.21875, 21241.857421875, -1.1083420515060425,
       -1.0984885692596436, 2.883861780166626, -1.663317322731018,
       276168.0, 35399.75, 0.062530517578125, 0.0249786376953125, -1.0,
       1.0, 13.0, 13.0, 1, 1, 0.0, 1170.9208984375, -1148.0504150390625,
       -863.2520751953125, -0.01893552392721176, -0.0006596783059649169,
       3.6064531803131104, 0.05354595556855202, 5359.7197265625,
       213.811508178710

In [31]:
l_features = ["lep_pt1", "lep_eta1", "lep_phi1", "lep_E1",
              "lep_pt2", "lep_eta2", "lep_phi2", "lep_E2",
              "jet_pt1", "jet_eta1", "jet_phi1", "jet_E1",
              "jet_pt2", "jet_eta2", "jet_phi2", "jet_E2",
              "met_et", "met_phi"]
df_flat_feat = df_flat[l_features]
df_flat_feat

,lep_pt1,lep_eta1,lep_phi1,lep_E1,lep_pt2,lep_eta2,lep_phi2,lep_E2,jet_pt1,jet_eta1,jet_phi1,jet_E1,jet_pt2,jet_eta2,jet_phi2,jet_E2,met_et,met_phi
entry,,,,,,,,,,,,,,,,,,
1,164413.218750,-1.108342,2.883862,276168.000000,21241.857422,-1.098489,-1.663317,35399.750000,142721.781250,0.401422,2.953223,154810.906250,114061.531250,0.508228,1.475421,129579.273438,333983.812500,-0.482479
5,180587.578125,-0.463239,-0.247670,200312.781250,26684.228516,0.287352,-1.374982,27793.505859,82331.773438,-1.115031,-1.161391,139869.750000,71367.703125,-0.410524,-2.380411,77931.460938,284773.593750,1.591610
9,150030.265625,0.111532,0.055983,150964.375000,72773.015625,-0.468784,-0.586932,80916.773438,278988.750000,-0.522298,2.226038,318263.812500,106945.687500,0.642534,-2.821163,130512.882812,173399.828125,-1.136152
10,122340.539062,0.907592,0.276549,176283.046875,78557.257812,1.810914,0.486651,246651.765625,117328.976562,1.227695,-2.760364,219045.546875,116146.171875,1.372407,2.804398,243967.875000,74956.648438,-1.217941
18,60599.914062,-0.374063,-0.775649,64889.234375,53151.660156,-1.095028,0.824327,88332.648438,73764.273438,-1.208271,1.705422,135326.484375,44259.128906,-1.848652,0.721330,144180.734375,166347.421875,-2.849924
19,273632.250000,-0.214895,-2.223884,279974.781250,100558.273438,0.177484,-1.894443,102146.304688,106235.695312,-0.576401,1.829600,124905.507812,85819.718750,-1.997697,-2.616994,322232.656250,375390.218750,0.568995
20,112955.703125,-0.807716,0.735955,151849.453125,38671.984375,-1.305332,-0.617317,76570.484375,82051.406250,-0.204724,-0.530093,83820.867188,73796.117188,0.423925,-1.209777,80816.945312,215259.578125,2.856133
22,87494.757812,-1.063262,-2.195217,141790.921875,50522.914062,-1.723425,-2.243865,146065.390625,374921.093750,-1.412252,0.675328,815333.187500,31686.982422,-0.726929,0.602897,40711.960938,322435.031250,-2.823792
24,102734.968750,1.211917,2.609928,187878.937500,11964.189453,-0.955894,-2.517672,17859.638672,76658.171875,1.500184,-2.090487,180415.671875,49931.570312,1.694330,-3.075476,140660.562500,192578.859375,0.130537


In [32]:
l_eventweights = ['mcWeight',
                  'scaleFactor_PILEUP',
                  'scaleFactor_ELE',
                  'scaleFactor_MUON',
                  'scaleFactor_BTAG',
                  'scaleFactor_LepTRIGGER']
df_flat[l_eventweights]

,mcWeight,scaleFactor_PILEUP,scaleFactor_ELE,scaleFactor_MUON,scaleFactor_BTAG,scaleFactor_LepTRIGGER
entry,,,,,,
1,1.000000,0.187531,1.000000,0.996780,0.983811,0.962223
5,1.000000,0.351687,0.948735,1.000000,1.224505,1.005820
9,1.000000,0.619412,0.992704,1.000000,1.006577,1.009360
10,1.000000,0.619412,0.962540,1.000000,0.985024,1.003060
18,1.000000,0.738662,0.970760,1.000000,0.981073,0.959720
19,1.000000,0.696077,1.000000,0.988170,0.998710,0.964819
20,1.000000,0.002209,0.922809,1.000000,0.999934,0.991800
22,1.000000,0.506784,1.000000,0.986359,0.998594,0.971463
24,1.000000,0.402058,1.000000,0.998355,0.999524,1.010198


In [35]:
s_eventweights = df_flat.scaleFactor_PILEUP * df_flat.scaleFactor_ELE * df_flat.scaleFactor_MUON * df_flat.scaleFactor_BTAG * df_flat.scaleFactor_LepTRIGGER
s_eventweights

entry
1     0.176953
5     0.410943
9     0.624729
10    0.589077
18    0.675156
19    0.662786
20    0.002022
22    0.484923
24    0.405298
25    0.494111
27    0.871776
29    1.067326
31    1.006157
36    1.050752
45    0.000000
46    0.002623
53    0.587862
59    0.749643
62    0.702410
63    0.003216
68    0.568526
69    0.694662
72    1.245357
75    1.603561
76    1.456053
78    1.193540
81    1.201368
82    1.096830
90    0.335493
91    0.483704
93    0.440975
95    0.597608
97    0.589372
98    0.597453
99    0.629909
dtype: float32

In [38]:
import numpy as np

In [41]:
s_ew = pd.Series(1, index=np.arange(len(df_flat)))
s_ew

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    1
21    1
22    1
23    1
24    1
25    1
26    1
27    1
28    1
29    1
30    1
31    1
32    1
33    1
34    1
dtype: int64

In [47]:
df_flat[l_eventweights].agg('prod', axis='columns')

entry
1     0.176953
5     0.410943
9     0.624729
10    0.589077
18    0.675156
19    0.662786
20    0.002022
22    0.484923
24    0.405298
25    0.494111
27    0.871776
29    1.067326
31    1.006157
36    1.050752
45    0.000000
46    0.002623
53    0.587862
59    1.025004
62    0.754287
63    0.003774
68    0.568526
69    0.700799
72    1.496964
75    1.603561
76    1.456053
78    1.193540
81    1.201368
82    1.286601
90    0.335493
91    0.483704
93    0.440975
95    0.597608
97    0.589372
98    0.597453
99    0.629909
dtype: float32

In [57]:
from sklearn.utils import shuffle
df_flat = shuffle(df_flat)
df_flat

,runNumber,eventNumber,channelNumber,mcWeight,scaleFactor_PILEUP,scaleFactor_ELE,scaleFactor_MUON,scaleFactor_PHOTON,scaleFactor_TAU,scaleFactor_BTAG,...,lep_ptcone301,lep_ptcone302,lep_etcone201,lep_etcone202,lep_trackd0pvunbiased1,lep_trackd0pvunbiased2,lep_tracksigd0pvunbiased1,lep_tracksigd0pvunbiased2,lep_pt_syst1,lep_pt_syst2
entry,,,,,,,,,,,,,,,,,,,,,
72,284500.0,1389.0,392302.0,1.202036,1.087349,0.959266,1.000000,1.0,1.0,1.187040,...,0.000000,0.000000,1291.300537,175.445923,0.017012,0.022125,2.433726,2.047694,618.045532,170.626801
76,284500.0,1689.0,392302.0,1.000000,1.554649,0.942289,1.000000,1.0,1.0,0.989065,...,0.000000,0.000000,1373.319702,686.182007,0.025357,-0.017326,3.853335,1.473655,3312.362305,2382.755859
31,284500.0,1388.0,392302.0,1.000000,1.087349,0.949683,1.000000,1.0,1.0,0.986760,...,1391.605957,0.000000,919.919556,-207.410889,0.023112,0.008304,3.035558,1.155709,2479.415039,227.007034
29,284500.0,1325.0,392302.0,1.000000,1.087349,1.000000,0.995709,1.0,1.0,0.981354,...,0.000000,1122.768677,-346.832153,248.944550,0.024602,0.018892,4.225675,2.342603,1919.752441,2507.538330
63,284500.0,1013.0,392302.0,1.173507,0.002793,0.936856,1.000000,1.0,1.0,1.234703,...,0.000000,0.000000,363.786041,796.404175,-0.003229,-0.026483,0.433195,1.673435,2501.526855,131.309402
97,284500.0,639.0,392302.0,1.000000,0.619412,0.976365,1.000000,1.0,1.0,0.982592,...,0.000000,2544.019531,921.706421,-68.240356,-0.000740,-0.011150,0.094051,1.202680,1317.100952,927.458313
10,284500.0,514.0,392302.0,1.000000,0.619412,0.962540,1.000000,1.0,1.0,0.985024,...,0.000000,0.000000,-1637.653564,660.079590,-0.099559,0.006394,5.177276,0.606597,892.472473,500.018707
59,284500.0,833.0,392302.0,1.367323,0.738662,1.000000,0.992713,1.0,1.0,0.984689,...,0.000000,3703.516113,-653.402405,996.530212,-0.003316,-0.004186,0.586645,0.642440,536.273193,549.743225
98,284500.0,671.0,392302.0,1.000000,0.619412,0.966530,1.000000,1.0,1.0,0.999290,...,0.000000,0.000000,-368.489136,-1012.728149,0.022690,0.034544,3.932348,4.023888,2778.503174,240.994370


In [58]:
ndarr_feat = df_flat.values*1
ndarr_feat

array([[284500.0, 1389.0, 392302.0, ..., 2.047694206237793,
        618.0455322265625, 170.62680053710938],
       [284500.0, 1689.0, 392302.0, ..., 1.4736549854278564,
        3312.3623046875, 2382.755859375],
       [284500.0, 1388.0, 392302.0, ..., 1.1557087898254395,
        2479.4150390625, 227.0070343017578],
       ...,
       [284500.0, 716.0, 392302.0, ..., 0.8828344345092773,
        450.6663513183594, 50.733856201171875],
       [284500.0, 1215.0, 392302.0, ..., 0.32590052485466003,
        1116.632568359375, 139.20654296875],
       [284500.0, 1654.0, 392302.0, ..., 1.1077462434768677,
        350.3843688964844, 220.31431579589844]], dtype=object)

In [59]:
ndarr_feat.shape

(35, 76)

In [60]:
len(ndarr_feat)

35

In [1]:
y_zeros = np.zeros(len(ndarr_feat))
y_ones = np.ones(len(ndarr_feat))
y = np.concatenate((y_zeros, y_ones), axis=0)
y.shape

NameError: name 'np' is not defined

In [12]:
store = pd.HDFStore('test.h5')

In [5]:
store['test_key'] = df_flat

In [6]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: test.h5

In [7]:
store.keys()

['/test_key']

In [8]:
df_test = store['test_key']
df_test

,runNumber,eventNumber,channelNumber,mcWeight,scaleFactor_PILEUP,scaleFactor_ELE,scaleFactor_MUON,scaleFactor_PHOTON,scaleFactor_TAU,scaleFactor_BTAG,...,trigDT,lep_n,met_et,met_phi,jet_n,photon_n,fatjet_n,tau_n,ditau_m,met_et_syst
entry,,,,,,,,,,,,,,,,,,,,,
0,284500,44,392302,1.000000,0.000000,0.975380,1.011483,1.0,1.0,0.997738,...,False,2,304174.156250,-0.155437,4,0,0,0,0.0,7923.125000
1,284500,130,392302,1.000000,0.187531,1.000000,0.996780,1.0,1.0,0.983811,...,False,2,333983.812500,-0.482479,2,0,0,0,0.0,4881.031250
2,284500,167,392302,1.000000,0.187531,0.895096,1.000000,1.0,1.0,0.998033,...,False,2,312675.625000,2.897666,2,0,0,0,0.0,15660.812500
3,284500,211,392302,1.000000,0.351687,1.000000,0.996622,1.0,1.0,1.186003,...,False,2,448053.250000,2.432551,2,0,0,0,0.0,593.437500
4,284500,230,392302,1.514916,0.351687,1.000000,0.990201,1.0,1.0,0.982900,...,False,2,211326.234375,-0.996418,3,0,0,0,0.0,17517.875000
5,284500,269,392302,1.000000,0.351687,0.948735,1.000000,1.0,1.0,1.224505,...,False,2,284773.593750,1.591610,4,0,0,0,0.0,10489.312500
6,284500,321,392302,1.000000,0.553627,0.957774,1.000000,1.0,1.0,1.014888,...,False,2,223925.453125,0.256444,3,0,0,0,0.0,148.937500
7,284500,345,392302,1.000000,0.553627,0.960924,1.000000,1.0,1.0,0.999208,...,False,2,327234.250000,2.241096,1,0,0,0,0.0,3070.156250
8,284500,429,392302,1.000000,0.553627,1.000000,0.997205,1.0,1.0,0.998448,...,False,2,426801.125000,-0.172173,3,0,0,0,0.0,9162.468750


In [11]:
len(df_flat.query('jet_n == 2').loc[:,'channelNumber'])

34

In [4]:
type(df_flat["channelNumber"][0])

numpy.int32

In [10]:
type(np.int32(infos["chargino500-neutralino2L2J100"]["DSID"]))

numpy.int32

In [11]:
infos["chargino500-neutralino2L2J100"]["DSID"]

392302

In [16]:
np.int32(392302) in df_flat["channelNumber"]

False

In [18]:
type(np.int32(1))

numpy.int32

In [19]:
df_flat["channelNumber"]

entry
0     392302
1     392302
2     392302
3     392302
4     392302
5     392302
6     392302
7     392302
8     392302
9     392302
10    392302
11    392302
12    392302
13    392302
14    392302
15    392302
16    392302
17    392302
18    392302
19    392302
20    392302
21    392302
22    392302
23    392302
24    392302
25    392302
26    392302
27    392302
28    392302
29    392302
       ...  
70    392302
71    392302
72    392302
73    392302
74    392302
75    392302
76    392302
77    392302
78    392302
79    392302
80    392302
81    392302
82    392302
83    392302
84    392302
85    392302
86    392302
87    392302
88    392302
89    392302
90    392302
91    392302
92    392302
93    392302
94    392302
95    392302
96    392302
97    392302
98    392302
99    392302
Name: channelNumber, Length: 100, dtype: int32

In [21]:
392302 in set(df_flat["channelNumber"])

True

In [22]:
import numpy as np
infos["chargino500-neutralino2L2J100"]["DSID"] in set(df_flat["channelNumber"])

True

In [1]:
import pandas as pd

In [2]:
pd.Series([1]*10)

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
dtype: int64

In [3]:
from infofile import infos

In [4]:
info = {} 
for key in infos.keys(): 
        ID = infos[key]['DSID']
        info[ID] = {} 
        info[ID]['xsec'] = infos[key]['xsec'] 
        info[ID]['sumw'] = infos[key]['sumw'] 
        info[ID]['events'] = infos[key]['events']
infos

{'WR2400NR1800': {'DSID': 302681,
  'events': 20000.0,
  'red_eff': 1,
  'sumw': 20000.0,
  'xsec': 0.031435},
 'WR3000NR1500': {'DSID': 302687,
  'events': 15000.0,
  'red_eff': 1,
  'sumw': 15000.0,
  'xsec': 0.018827},
 'WR3600NR1800': {'DSID': 302701,
  'events': 20000.0,
  'red_eff': 1,
  'sumw': 20000.0,
  'xsec': 0.0051743},
 'WR4200NR2100': {'DSID': 302708,
  'events': 19000.0,
  'red_eff': 1,
  'sumw': 19000.0,
  'xsec': 0.0014895},
 'WR5000NR2500': {'DSID': 309070,
  'events': 19000.0,
  'red_eff': 1,
  'sumw': 19000.0,
  'xsec': 0.00028772},
 'Zprime2000ee': {'DSID': 301215,
  'events': 1,
  'red_eff': 1,
  'sumw': 19800,
  'xsec': 0.0088432},
 'Zprime3000ee': {'DSID': 301216,
  'events': 1,
  'red_eff': 1,
  'sumw': 19600,
  'xsec': 0.00080617},
 'Zprime4000ee': {'DSID': 301217,
  'events': 1,
  'red_eff': 1,
  'sumw': 19800,
  'xsec': 0.00010351},
 'Zprime5000ee': {'DSID': 301218,
  'events': 1,
  'red_eff': 1,
  'sumw': 18000,
  'xsec': 1.8319e-05},
 'Zprime2000mumu': {'D

In [5]:
10.6*0.025481788*1e3/10419.6442093

0.025922857573094232

In [6]:
import pandas as pd
df_info = pd.DataFrame(info).T
df_info

,events,sumw,xsec
302681,20000.0,2.000000e+04,0.031435
302687,15000.0,1.500000e+04,0.018827
302701,20000.0,2.000000e+04,0.005174
302708,19000.0,1.900000e+04,0.001489
309070,19000.0,1.900000e+04,0.000288
301215,1.0,1.980000e+04,0.008843
301216,1.0,1.960000e+04,0.000806
301217,1.0,1.980000e+04,0.000104
301218,1.0,1.800000e+04,0.000018
301220,1.0,9.830000e+05,0.008880


In [7]:
import numpy as np
df_test = pd.DataFrame(data=np.arange(9).reshape((3,3)), columns=["A", "B", "C"])
df_test

,A,B,C
0,0,1,2
1,3,4,5
2,6,7,8


In [8]:
df_test.loc[:1,"B"] = 10
df_test

,A,B,C
0,0,10,2
1,3,10,5
2,6,7,8


In [9]:
df_test["index"] = df_test.index
df_test

,A,B,C,index
0,0,10,2,0
1,3,10,5,1
2,6,7,8,2


In [10]:
df_test = df_test.set_index("B")
df_test

,A,C,index
B,,,
10,0,2,0
10,3,5,1
7,6,8,2


In [11]:
df_test = pd.concat([df_test, pd.DataFrame(columns=["events", "sumw", "xsec"])], axis=1)
df_test

,A,C,index,events,sumw,xsec
B,,,,,,
10,0,2,0,NaN,NaN,NaN
10,3,5,1,NaN,NaN,NaN
7,6,8,2,NaN,NaN,NaN


In [13]:
df_test.xsec.isna()

B
10    True
10    True
7     True
Name: xsec, dtype: bool

In [77]:
df_test.index

Int64Index([10, 10, 7], dtype='int64', name='B')

In [78]:
df_test.loc[10, ["events", "sumw", "xsec"]] = [1, 2, 3]
df_test

,A,C,index,events,sumw,xsec
B,,,,,,
10,0,2,0,1,2,3
10,3,5,1,1,2,3
7,6,8,2,NaN,NaN,NaN


In [79]:
df_test.insert(1, "B", df_test.index)
df_test

,A,B,C,index,events,sumw,xsec
B,,,,,,,
10,0,10,2,0,1,2,3
10,3,10,5,1,1,2,3
7,6,7,8,2,NaN,NaN,NaN


In [67]:
df_test = df_test.set_index("index")
df_test

,A,B,C,events,sumw,xsec
index,,,,,,
0,0,10,2,1,2,3
1,3,10,5,1,2,3
2,6,7,8,NaN,NaN,NaN


In [81]:
df_test["B/C"] = df_test["B"]/df_test["C"]
df_test

,A,B,C,index,events,sumw,xsec,B/C
B,,,,,,,,
10,0,10,2,0,1,2,3,5.000
10,3,10,5,1,1,2,3,2.000
7,6,7,8,2,NaN,NaN,NaN,0.875


In [82]:
s_test = pd.Series([3, 2, 1], index=[10, 10, 7])
s_test

10    3
10    2
7     1
dtype: int64

In [83]:
df_test["series"] = s_test
df_test

,A,B,C,index,events,sumw,xsec,B/C,series
B,,,,,,,,,
10,0,10,2,0,1,2,3,5.000,3
10,3,10,5,1,1,2,3,2.000,2
7,6,7,8,2,NaN,NaN,NaN,0.875,1


In [42]:
df_test.loc[5, ["events", "sumw", "xsec"]] = [1, 1, 1]
df_test

,A,B,C,events,sumw,xsec
idx,,,,,,
5,0.0,1.0,2.0,1,1,1
5,3.0,4.0,5.0,1,1,1
5,6.0,7.0,8.0,1,1,1
5,NaN,NaN,NaN,1,1,1


In [67]:
i = None
1 is None

False

In [68]:
j = 1
1 is j

True